# Getting the Git

In [2]:
import os
%matplotlib inline
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib as mpl
import time
import sys
os.environ['PATH'] += ";C\Program Files\Git\\bin"

toolbar_width = 40

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1)) 

[                                        ]

In [ ]:
#%%bash
#rm -rf fec-data
#git clone git@github.com:jbisbee1/fec-data.git

In [3]:
%%bash
git log

commit fd0054cc07e492fb2f0aa3389652e3f5f8635dd5
Author: jbisbee1 <jimbisbee@yahoo.com>
Date:   Fri Jul 22 09:27:10 2016 -0400

    Implementing fuzzy wuzzy and using it to create Qualtrics surveys for manual checking.

commit 4cd451b67957990f506074a7f38e200ceb053862
Author: jbisbee1 <jimbisbee@yahoo.com>
Date:   Thu Jul 21 16:37:22 2016 -0400

    Updated the readme.md file to remind myself of fuzzyfuzzy.

commit 3f7beb7944abf2f86654fc6b9c15fcfdb61f4091
Author: jbisbee1 <jimbisbee@yahoo.com>
Date:   Thu Jul 21 12:15:12 2016 -0400

    Updated the readme.md file with a brief description of the purpose for this repository.


# Opening some text data

In [4]:
recipients = pd.read_csv("../../earmarks/recipient.csv")
recipients.head()

,id,earmark_id,raw_recipient,standardized_recipient
0,1,6,Ocanit,NaN
1,3,12,Rescue Technolgies Corporation,NaN
2,4,18,APHIS Hawaii,NaN
3,5,25,University of Hawaii - University Affliated Re...,NaN
4,6,32,Alaka'i Consulting and Engineering,NaN


# Cleaning the data

In [5]:
recraw = recipients['raw_recipient']
rectokens=recraw.str.split()
totaltokens = len(rectokens)

In [6]:
recclean = recraw.str.lower()
recclean = recclean.str.replace(',',' ',case=False)
recclean = recclean.str.replace('\&',' and ',case=False)
recclean = recclean.str.replace('-',' ',case=False)
recclean = recclean.str.replace('\'','',case=False)
recclean = recclean.str.replace('.','')
recclean = recclean.str.replace('[ ]{2,}',' ',case=False)
recclean = recclean.str.normalize('NFC')

# Trying to Estimate Text Similarities

In [7]:
test = recclean.str.split(expand = True)
type(test[0])
#test['universities'] = test[0].apply(lambda x: x.count('university'))
#test.head()

pandas.core.series.Series

In [8]:
recclean.str.extractall('(university)|(inc)')[:5]

0    1
  match                 
3 0      university  NaN
  1      university  NaN
5 0             NaN  inc
6 0      university  NaN
7 0             NaN  inc

In [9]:
recclean.str.count(" university").sum() #This is strange...two similar methods yield slightly different counts

725.0

In [10]:
recclean.str.contains(' university').sum()

724

In [11]:
rectokens = recclean.str.split()

# It seems there are packages already developed

In [ ]:
%%bash
pip install fuzzywuzzy

In [14]:
%%bash
pip install "C:/Users/jhb362/Dropbox/Programs/python_Levenshtein-0.12.0-cp35-none-win_amd64.whl"

Processing c:\users\jhb362\dropbox\programs\python_levenshtein-0.12.0-cp35-none-win_amd64.whl


In [15]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [16]:
recclean.head()

0                                               ocanit
1                       rescue technolgies corporation
2                                         aphis hawaii
3    university of hawaii university affliated rese...
4                    alakai consulting and engineering
Name: raw_recipient, dtype: object

In [17]:
uniques = recclean.drop_duplicates()
select = [isinstance(e,str) for e in uniques]
uniques2 = uniques[select]

select = [isinstance(e,str) for e in recclean]
recclean2 = recclean[select]
select[:3]

[True, True, True]

In [18]:
pairings={}
for rec in recclean2:
    pairings[rec]=process.extractBests(rec,uniques2,scorer=fuzz.token_set_ratio,score_cutoff=92)
    # update the bar
    sys.stdout.write("-")
    sys.stdout.flush()

sys.stdout.write("\n")

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
mask = [list(pairings.items())[i] for i in range(0,len(list(pairings.items()))) if len(list(pairings.items())[i][1]) > 1]

# Write a .txt file in the format for importing to Qualtrics

In [20]:
counter = 0
for j in range(0,len(mask),10):
    print("Block Test: %s" % j)
    for k in range(0,len(mask[counter])):
        print("Advanced Choices: %s" % mask[counter][0] + " %s" % counter)
        for i in range(1,len(mask[counter][1])):
            print("Choice: %s" % i + " %s" % mask[counter][1][i][0])
        counter += 1

Block Test: 0
Advanced Choices: university of north dakota grand forks nd 0
Choice: 1 university
Choice: 2 university of north dakota grand forks nd
Advanced Choices: goodrich isr systems princeton 1
Choice: 1 goodrich isr systems princeton (holt)
Choice: 2 goodrich isr systems princeton
Block Test: 10
Advanced Choices: oceanic institute 2
Choice: 1 oceanic institute waimanalu hi
Advanced Choices: lockheed martin missiles and fire control 3
Choice: 1 lockheed martin
Block Test: 20
Advanced Choices: university of nebraska medical center omaha ne 4
Choice: 1 university of nebraska omaha
Choice: 2 university of nebraska medical center
Choice: 3 university
Choice: 4 university of nebraska medical center omaha ne
Advanced Choices: appalachian mountain club boston ma 5
Choice: 1 appalachian mountain club boston ma
Block Test: 30
Advanced Choices: iowa state university ames ia 6
Choice: 1 ames iowa
Choice: 2 university
Choice: 3 iowa state university ames ia
Advanced Choices: university of al

In [21]:
survey_file = open("survey-test.txt","w")
survey_file.write("[[AdvancedFormat]] \n\n")
counter = 0
for j in range(0,len(mask),10):
    survey_file.write("[[Block: Firm Names - %s" % j + "]]\n")
    for k in range(0,len(mask[counter])):
        survey_file.write("[[Question:MC:MultipleAnswer:Vertical]] \n \
        %s" % mask[counter][0].upper() + "\n\n")
        survey_file.write("[[AdvancedChoices]] \n")
        for i in range(1,len(mask[counter][1])):
            survey_file.write("[[Choice: %s" % i +"]] \n \
            %s" % mask[counter][1][i][0].upper() +'\n')
        counter +=1
        survey_file.write("\n")
survey_file.close()

In [ ]:
%%bash
git add fec-string-prep.ipynb
git add survey-test.txt
git status
git commit -m "Updated Qualtrics import .txt file with earmarks firms."